##Imports and Data extracts from GitHub-Archive

In [2]:
import requests
import gzip
import json
import traceback


from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import types as T

###### Note: Since there was memory constraint in DataBricks Community environment, only 12 hours of data from 01-01-2015 is used for further analysis.

In [4]:
events_list = []


def extract_data():

  for hour in range(12):
    url = 'http://data.gharchive.org/2015-01-01-'+str(hour)+'.json.gz'
    r = requests.get(url)
    file_path = "./"+url.split("/")[-1]

    with open(file_path, 'wb') as f:
        f.write(r.content)

    with gzip.open(file_path, 'rb') as f:
        for line in f:
            event = json.loads(line.decode("utf-8"))
            events_list.append(event)
  return


In [5]:
extract_data()

In [6]:
print(type(events_list))
print(len(events_list))
print(type(events_list[0]))

<class 'list'>
82157
<class 'dict'>

###### Combined all 12 json.gz files into a single json and dumped in DBFS (DataBricks File System).

In [8]:
file_path = "/dbfs/mnt/2015-01-01.json"
    
with open(file_path, 'w+') as fout:
  json.dump(events_list, fout)

###### Loaded the json as PySpark Dataframe: master_df. It contains raw json dump.

In [10]:
file_path = "dbfs:/mnt/2015-01-01.json"
master_df = spark.read.format("json").load(file_path)
#master_df.show()

In [11]:
master_df.printSchema()

root
-- actor: struct (nullable = true)
 |-- avatar_url: string (nullable = true)
 |-- gravatar_id: string (nullable = true)
 |-- id: long (nullable = true)
 |-- login: string (nullable = true)
 |-- url: string (nullable = true)
-- created_at: string (nullable = true)
-- id: string (nullable = true)
-- org: struct (nullable = true)
 |-- avatar_url: string (nullable = true)
 |-- gravatar_id: string (nullable = true)
 |-- id: long (nullable = true)
 |-- login: string (nullable = true)
 |-- url: string (nullable = true)
-- payload: struct (nullable = true)
 |-- action: string (nullable = true)
 |-- before: string (nullable = true)
 |-- comment: struct (nullable = true)
 | |-- _links: struct (nullable = true)
 | | |-- html: struct (nullable = true)
 | | | |-- href: string (nullable = true)
 | | |-- pull_request: struct (nullable = true)
 | | | |-- href: string (nullable = true)
 | | |-- self: struct (nullable = true)
 | | | |-- href: string (nullable = true)
 | |-- body: string (nullable = true)
 | |-- commit_id: string (nullable = true)
 | |-- created_at: string (nullable = true)
 | |-- diff_hunk: string (nullable = true)
 | |-- html_url: string (nullable = true)
 | |-- id: long (nullable = true)
 | |-- issue_url: string (nullable = true)
 | |-- line: long (nullable = true)
 | |-- original_commit_id: string (nullable = true)
 | |-- original_position: long (nullable = true)
 | |-- path: string (nullable = true)
 | |-- position: long (nullable = true)
 | |-- pull_request_url: string (nullable = true)
 | |-- updated_at: string (nullable = true)
 | |-- url: string (nullable = true)
 | |-- user: struct (nullable = true)
 | | |-- avatar_url: string (nullable = true)
 | | |-- events_url: string (nullable = true)
 | | |-- followers_url: string (nullable = true)
 | | |-- following_url: string (nullable = true)
 | | |-- gists_url: string (nullable = true)
 | | |-- gravatar_id: string (nullable = true)
 | | |-- html_url: string (nullable = true)
 | | |-- id: long (nullable = true)
 | | |-- login: string (nullable = true)
 | | |-- organizations_url: string (nullable = true)
 | | |-- received_events_url: string (nullable = true)
 | | |-- repos_url: string (nullable = true)
 | | |-- site_admin: boolean (nullable = true)
 | | |-- starred_url: string (nullable = true)
 | | |-- subscriptions_url: string (nullable = true)
 | | |-- type: string (nullable = true)
 | | |-- url: string (nullable = true)
 |-- commits: array (nullable = true)
 | |-- element: struct (containsNull = true)
 | | |-- author: struct (nullable = true)
 | | | |-- email: string (nullable = true)
 | | | |-- name: string (nullable = true)
 | | |-- distinct: boolean (nullable = true)
 | | |-- message: string (nullable = true)
 | | |-- sha: string (nullable = true)
 | | |-- url: string (nullable = true)
 |-- description: string (nullable = true)
 |-- distinct_size: long (nullable = true)
 |-- forkee: struct (nullable = true)
 | |-- archive_url: string (nullable = true)
 | |-- assignees_url: string (nullable = true)
 | |-- blobs_url: string (nullable = true)
 | |-- branches_url: string (nullable = true)
 | |-- clone_url: string (nullable = true)
 | |-- collaborators_url: string (nullable = true)
 | |-- comments_url: string (nullable = true)
 | |-- commits_url: string (nullable = true)
 | |-- compare_url: string (nullable = true)
 | |-- contents_url: string (nullable = true)
 | |-- contributors_url: string (nullable = true)
 | |-- created_at: string (nullable = true)
 | |-- default_branch: string (nullable = true)
 | |-- description: string (nullable = true)
 | |-- downloads_url: string (nullable = true)
 | |-- events_url: string (nullable = true)
 | |-- fork: boolean (nullable = true)
 | |-- forks: long (nullable = true)
 | |-- forks_count: long (nullable = true)
 | |-- forks_url: string (nullable = true)
 | |-- full_name: string (nullable = true)
 | |-- git_commits_url: string (nullable = true)
 | |-- git_refs_url: string (nullable = true)
 | |-- git_tags_url: string (nullable = tru

In [12]:
master_df.count()

Out[52]: 82157

## DataFrame 1 : main_df

In [14]:
main_df = master_df.select(F.col("actor.id").alias("actor_id"), 
                           F.col("org.id").alias("org_id"), 
                           F.col("id").alias("event_id"), 
                           F.col("created_at"), 
                           F.col("repo.id").alias("repo_id"), 
                           F.col("type").alias("event_type"))
main_df.show()

+--------+--------+----------+--------------------+--------+--------------------+
actor_id| org_id| event_id| created_at| repo_id| event_type|
+--------+--------+----------+--------------------+--------+--------------------+
 9152315| null|2489368070|2015-01-01T00:00:00Z|28635890| PushEvent|
 5581438| null|2489368072|2015-01-01T00:00:00Z|26392647| PushEvent|
 6352424| null|2489368089|2015-01-01T00:00:01Z|28677542| CreateEvent|
 66577| 82592|2489368095|2015-01-01T00:00:01Z| 5152285| PushEvent|
 9221683| null|2489368104|2015-01-01T00:00:03Z|28520835| PushEvent|
 1174658| null|2489368109|2015-01-01T00:00:03Z|25334511| PushEvent|
 2875833| null|2489368114|2015-01-01T00:00:03Z|21995378| PushEvent|
 1317792| null|2489368113|2015-01-01T00:00:03Z|15526570| PushEvent|
 1373703| null|2489368118|2015-01-01T00:00:03Z|20268125| PushEvent|
 6329667| null|2489368129|2015-01-01T00:00:04Z|27156833| CreateEvent|
 2676867| null|2489368130|2015-01-01T00:00:05Z| 3633660| PushEvent|
 5545980| null|2489368134|2015-01-01T00:00:05Z|27858779| PushEvent|
 4070158| null|2489368137|2015-01-01T00:00:05Z|20469468| PushEvent|
 3495129| null|2489368138|2015-01-01T00:00:05Z|24147122| PushEvent|
 4443263| null|2489368140|2015-01-01T00:00:05Z|28677492| PushEvent|
 21826| null|2489368145|2015-01-01T00:00:05Z|20117626| PushEvent|
 3952668| 9995739|2489368151|2015-01-01T00:00:06Z|27288715| IssueCommentEvent|
 2112033| null|2489368159|2015-01-01T00:00:08Z|28676854| PushEvent|
 3388704|10214048|2489368161|2015-01-01T00:00:08Z|28109544|PullRequestReview...|
 1987779| null|2489368162|2015-01-01T00:00:08Z| 5089162| PushEvent|
+--------+--------+----------+--------------------+--------+--------------------+
only showing top 20 rows

##### Data wrangling & exploration. Changed the datatypes of a few columns, split the timestamp into hour column, etc.

In [17]:
main_df = main_df.withColumn("created_time", F.split(F.col("created_at"), "T").getItem(1))
main_df = main_df.withColumn("created_hour", F.split(F.col("created_time"), ":").getItem(0)).drop("created_at").drop("created_time")
main_df = main_df.withColumn("actor_id", main_df["actor_id"].cast(T.LongType())).withColumn("org_id", main_df["org_id"].cast(T.LongType())).withColumn("event_id", main_df["event_id"].cast(T.LongType())).withColumn("repo_id", main_df["repo_id"].cast(T.LongType())).withColumn("created_hour", main_df["created_hour"].cast(T.IntegerType()))
print(main_df.show())
print(main_df.printSchema())

+--------+--------+----------+--------+--------------------+------------+
actor_id| org_id| event_id| repo_id| event_type|created_hour|
+--------+--------+----------+--------+--------------------+------------+
 9152315| null|2489368070|28635890| PushEvent| 0|
 5581438| null|2489368072|26392647| PushEvent| 0|
 6352424| null|2489368089|28677542| CreateEvent| 0|
 66577| 82592|2489368095| 5152285| PushEvent| 0|
 9221683| null|2489368104|28520835| PushEvent| 0|
 1174658| null|2489368109|25334511| PushEvent| 0|
 2875833| null|2489368114|21995378| PushEvent| 0|
 1317792| null|2489368113|15526570| PushEvent| 0|
 1373703| null|2489368118|20268125| PushEvent| 0|
 6329667| null|2489368129|27156833| CreateEvent| 0|
 2676867| null|2489368130| 3633660| PushEvent| 0|
 5545980| null|2489368134|27858779| PushEvent| 0|
 4070158| null|2489368137|20469468| PushEvent| 0|
 3495129| null|2489368138|24147122| PushEvent| 0|
 4443263| null|2489368140|28677492| PushEvent| 0|
 21826| null|2489368145|20117626| PushEvent| 0|
 3952668| 9995739|2489368151|27288715| IssueCommentEvent| 0|
 2112033| null|2489368159|28676854| PushEvent| 0|
 3388704|10214048|2489368161|28109544|PullRequestReview...| 0|
 1987779| null|2489368162| 5089162| PushEvent| 0|
+--------+--------+----------+--------+--------------------+------------+
only showing top 20 rows

None
root
-- actor_id: long (nullable = true)
-- org_id: long (nullable = true)
-- event_id: long (nullable = true)
-- repo_id: long (nullable = true)
-- event_type: string (nullable = true)
-- created_hour: integer (nullable = true)

None

In [18]:
main_df.describe().show()

+-------+------------------+------------------+-------------------+--------------------+------------------+-----------------+
summary| actor_id| org_id| event_id| repo_id| event_type| created_hour|
+-------+------------------+------------------+-------------------+--------------------+------------------+-----------------+
 count| 82157| 19986| 82157| 82157| 82157| 82157|
 mean| 3729451.807064523| 4549156.420394276|2.489481050832467E9|2.1054022713779714E7| null|5.494991297150578|
 stddev|3405710.6823425996|3504901.8525804924| 59071.733887044255| 8920119.493090013| null|3.527698452568258|
 min| 7| 81| 2489368070| 27|CommitCommentEvent| 0|
 max| 10363807| 10363764| 2489582585| 28685790| WatchEvent| 11|
+-------+------------------+------------------+-------------------+--------------------+------------------+-----------------+

###### Imputed missing values from main_df column instead of Null or NaNs.

In [20]:
main_df = main_df.fillna({'actor_id': -1, 'org_id': -1, 'event_id': -1, 'repo_id': -1, 'event_type': "Missing", 'created_hour': -1})
main_df.show()

+--------+--------+----------+--------+--------------------+------------+
actor_id| org_id| event_id| repo_id| event_type|created_hour|
+--------+--------+----------+--------+--------------------+------------+
 9152315| -1|2489368070|28635890| PushEvent| 0|
 5581438| -1|2489368072|26392647| PushEvent| 0|
 6352424| -1|2489368089|28677542| CreateEvent| 0|
 66577| 82592|2489368095| 5152285| PushEvent| 0|
 9221683| -1|2489368104|28520835| PushEvent| 0|
 1174658| -1|2489368109|25334511| PushEvent| 0|
 2875833| -1|2489368114|21995378| PushEvent| 0|
 1317792| -1|2489368113|15526570| PushEvent| 0|
 1373703| -1|2489368118|20268125| PushEvent| 0|
 6329667| -1|2489368129|27156833| CreateEvent| 0|
 2676867| -1|2489368130| 3633660| PushEvent| 0|
 5545980| -1|2489368134|27858779| PushEvent| 0|
 4070158| -1|2489368137|20469468| PushEvent| 0|
 3495129| -1|2489368138|24147122| PushEvent| 0|
 4443263| -1|2489368140|28677492| PushEvent| 0|
 21826| -1|2489368145|20117626| PushEvent| 0|
 3952668| 9995739|2489368151|27288715| IssueCommentEvent| 0|
 2112033| -1|2489368159|28676854| PushEvent| 0|
 3388704|10214048|2489368161|28109544|PullRequestReview...| 0|
 1987779| -1|2489368162| 5089162| PushEvent| 0|
+--------+--------+----------+--------+--------------------+------------+
only showing top 20 rows

In [21]:
main_df.describe().show()

+-------+------------------+------------------+-------------------+--------------------+------------------+-----------------+
summary| actor_id| org_id| event_id| repo_id| event_type| created_hour|
+-------+------------------+------------------+-------------------+--------------------+------------------+-----------------+
 count| 82157| 82157| 82157| 82157| 82157| 82157|
 mean| 3729451.807064523|1106654.0653504876|2.489481050832467E9|2.1054022713779714E7| null|5.494991297150578|
 stddev|3405710.6823425996| 2607289.147978606| 59071.733887044255| 8920119.493090013| null|3.527698452568258|
 min| 7| -1| 2489368070| 27|CommitCommentEvent| 0|
 max| 10363807| 10363764| 2489582585| 28685790| WatchEvent| 11|
+-------+------------------+------------------+-------------------+--------------------+------------------+-----------------+

##### Data Aggregation on main_df

In [23]:
print("Unique Actors: ", main_df.select('actor_id').distinct().count())
print("Unique Orgs: ", main_df.select('org_id').distinct().count())
print("Unique Repos: ", main_df.select('repo_id').distinct().count())
print("Unique Event Types: ", main_df.select('event_type').distinct().count())
event_types = [i for i in main_df.select('event_type').distinct().collect()]
print(event_types)

Unique Actors: 24323
Unique Orgs: 4159
Unique Repos: 29102
Unique Event Types: 14
[Row(event_type='PushEvent'), Row(event_type='GollumEvent'), Row(event_type='ReleaseEvent'), Row(event_type='CommitCommentEvent'), Row(event_type='CreateEvent'), Row(event_type='PullRequestReviewCommentEvent'), Row(event_type='IssueCommentEvent'), Row(event_type='DeleteEvent'), Row(event_type='IssuesEvent'), Row(event_type='ForkEvent'), Row(event_type='PublicEvent'), Row(event_type='MemberEvent'), Row(event_type='WatchEvent'), Row(event_type='PullRequestEvent')]

In [24]:
main_df.groupby("event_type").agg(F.count("event_id")).show()

+--------------------+---------------+
 event_type|count(event_id)|
+--------------------+---------------+
 PushEvent| 46919|
 GollumEvent| 924|
 ReleaseEvent| 297|
 CommitCommentEvent| 507|
 CreateEvent| 8632|
PullRequestReview...| 583|
 IssueCommentEvent| 5273|
 DeleteEvent| 1245|
 IssuesEvent| 3206|
 ForkEvent| 2817|
 PublicEvent| 71|
 MemberEvent| 169|
 WatchEvent| 8525|
 PullRequestEvent| 2989|
+--------------------+---------------+

In [25]:
print((main_df.filter(main_df.org_id != -1).select('actor_id').distinct().count()/main_df.select('actor_id').distinct().count())*100)

28.623113925091477

In [26]:
print((main_df.filter(main_df.org_id != -1).select('repo_id').distinct().count()/main_df.select('repo_id').distinct().count())*100)

21.946945227132154

## DataFrame 2: org_df

In [28]:
org_df = master_df.select(F.col("org.id"), F.col("org.login"), F.col("org.url")).dropDuplicates(["id"])

###### cleaning data and imputing missing values of org_df

In [30]:
org_df.printSchema()

root
-- id: long (nullable = true)
-- login: string (nullable = true)
-- url: string (nullable = true)

In [31]:
org_df.describe().show()

+-------+------------------+-------------------+--------------------+
summary| id| login| url|
+-------+------------------+-------------------+--------------------+
 count| 4158| 4158| 4158|
 mean| 4815555.561327562| 7.409327425E7| null|
 stddev|3436498.7002195385|9.855969120977792E7| null|
 min| 81| 01Studio|https://api.githu...|
 max| 10363764| zynga|https://api.githu...|
+-------+------------------+-------------------+--------------------+

In [32]:
org_df.sample(fraction = 0.010).show()

+--------+--------------------+--------------------+
 id| login| url|
+--------+--------------------+--------------------+
 6384029| easy-development|https://api.githu...|
 2109913| opencog|https://api.githu...|
 9652408| NTU-GameProgramming|https://api.githu...|
 2550273| mpv-player|https://api.githu...|
 9004206| BigSense|https://api.githu...|
 919335| MassiveCraft|https://api.githu...|
 4206165| M-R-K-Development|https://api.githu...|
 2306807| copy-app|https://api.githu...|
10328926| BJUT-Agile-Group|https://api.githu...|
 1562882| Roll20|https://api.githu...|
 1561857| FarsetLabs|https://api.githu...|
 7538207| JOATLabs|https://api.githu...|
 605176| Nextdoor|https://api.githu...|
 7356072| SynapLink|https://api.githu...|
 7413696| MrManhattan|https://api.githu...|
 7479435| Wohlhabend-Networks|https://api.githu...|
 1432696| ftlabs|https://api.githu...|
 3239934| x-tag|https://api.githu...|
 7001137|proyectos-mejoran...|https://api.githu...|
 517239| mysterioustrousers|https://api.githu...|
+--------+--------------------+--------------------+
only showing top 20 rows

In [33]:
org_df = org_df.fillna({'id': -1, 'login': "Missing", 'url': "Missing"})

## DataFrame 3: payload_df

In [35]:
payload_df = master_df.select(F.col("id").alias("event_id"), F.col("payload.*"))
payload_df.printSchema()

root
-- event_id: string (nullable = true)
-- action: string (nullable = true)
-- before: string (nullable = true)
-- comment: struct (nullable = true)
 |-- _links: struct (nullable = true)
 | |-- html: struct (nullable = true)
 | | |-- href: string (nullable = true)
 | |-- pull_request: struct (nullable = true)
 | | |-- href: string (nullable = true)
 | |-- self: struct (nullable = true)
 | | |-- href: string (nullable = true)
 |-- body: string (nullable = true)
 |-- commit_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- diff_hunk: string (nullable = true)
 |-- html_url: string (nullable = true)
 |-- id: long (nullable = true)
 |-- issue_url: string (nullable = true)
 |-- line: long (nullable = true)
 |-- original_commit_id: string (nullable = true)
 |-- original_position: long (nullable = true)
 |-- path: string (nullable = true)
 |-- position: long (nullable = true)
 |-- pull_request_url: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- url: string (nullable = true)
 |-- user: struct (nullable = true)
 | |-- avatar_url: string (nullable = true)
 | |-- events_url: string (nullable = true)
 | |-- followers_url: string (nullable = true)
 | |-- following_url: string (nullable = true)
 | |-- gists_url: string (nullable = true)
 | |-- gravatar_id: string (nullable = true)
 | |-- html_url: string (nullable = true)
 | |-- id: long (nullable = true)
 | |-- login: string (nullable = true)
 | |-- organizations_url: string (nullable = true)
 | |-- received_events_url: string (nullable = true)
 | |-- repos_url: string (nullable = true)
 | |-- site_admin: boolean (nullable = true)
 | |-- starred_url: string (nullable = true)
 | |-- subscriptions_url: string (nullable = true)
 | |-- type: string (nullable = true)
 | |-- url: string (nullable = true)
-- commits: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- author: struct (nullable = true)
 | | |-- email: string (nullable = true)
 | | |-- name: string (nullable = true)
 | |-- distinct: boolean (nullable = true)
 | |-- message: string (nullable = true)
 | |-- sha: string (nullable = true)
 | |-- url: string (nullable = true)
-- description: string (nullable = true)
-- distinct_size: long (nullable = true)
-- forkee: struct (nullable = true)
 |-- archive_url: string (nullable = true)
 |-- assignees_url: string (nullable = true)
 |-- blobs_url: string (nullable = true)
 |-- branches_url: string (nullable = true)
 |-- clone_url: string (nullable = true)
 |-- collaborators_url: string (nullable = true)
 |-- comments_url: string (nullable = true)
 |-- commits_url: string (nullable = true)
 |-- compare_url: string (nullable = true)
 |-- contents_url: string (nullable = true)
 |-- contributors_url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- default_branch: string (nullable = true)
 |-- description: string (nullable = true)
 |-- downloads_url: string (nullable = true)
 |-- events_url: string (nullable = true)
 |-- fork: boolean (nullable = true)
 |-- forks: long (nullable = true)
 |-- forks_count: long (nullable = true)
 |-- forks_url: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- git_commits_url: string (nullable = true)
 |-- git_refs_url: string (nullable = true)
 |-- git_tags_url: string (nullable = true)
 |-- git_url: string (nullable = true)
 |-- has_downloads: boolean (nullable = true)
 |-- has_issues: boolean (nullable = true)
 |-- has_pages: boolean (nullable = true)
 |-- has_wiki: boolean (nullable = true)
 |-- homepage: string (nullable = true)
 |-- hooks_url: string (nullable = true)
 |-- html_url: string (nullable = true)
 |-- id: long (nullable = true)
 |-- issue_comment_url: string (nullable = true)
 |-- issue_events_url: string (nullable = true)
 |-- issues_url: string (nullable = true)
 |-- keys_url: string (nullable = true)
 |-- labels_url: string (nullable = true)
 |-- language: string (nullable = true)
 |-- languages_url: string (nullable = true)
 |

##### select the fields of interest from payload_df and write them to a seperate dataframe

In [37]:
payload_df.show()

+----------+-------+--------------------+--------------------+--------------------+--------------------+-------------+------+--------------------+--------------------+-------------+------+------+-----+--------------------+---------+-----------+--------------------+--------+-------+----+
 event_id| action| before| comment| commits| description|distinct_size|forkee| head| issue|master_branch|member|number|pages| pull_request| push_id|pusher_type| ref|ref_type|release|size|
+----------+-------+--------------------+--------------------+--------------------+--------------------+-------------+------+--------------------+--------------------+-------------+------+------+-----+--------------------+---------+-----------+--------------------+--------+-------+----+
2489368070| null|86ffa724b4d70fce4...| null|[[[da8d7d1118ca5b...| null| 1| null|a9b22a6d80c1e0bb4...| null| null| null| null| null| null|536740396| null| refs/heads/master| null| null| 1|
2489368072| null|d5a69a84e4d95c1c2...| null|[[[61dcbbf08c9f7a...| null| 1| null|56688cc528224d406...| null| null| null| null| null| null|536740397| null| refs/heads/master| null| null| 1|
2489368089| null| null| null| null|Making an HTML5 g...| null| null| null| null| master| null| null| null| null| null| user| master| branch| null|null|
2489368095| null|8590ddf9fe54fe27d...| null|[[[9ec2b9d5f2203d...| null| 1| null|bc9aa8a0c999232d2...| null| null| null| null| null| null|536740405| null|refs/heads/jw/web...| null| null| 8|
2489368104| null|bc6e2f2a7d2095f88...| null|[[[0b727f1f5f209f...| null| 1| null|97a402b54bfd562b8...| null| null| null| null| null| null|536740413| null| refs/heads/master| null| null| 1|
2489368109| null|2d7a0159d00b28343...| null|[[[50f3f01caa0536...| null| 1| null|e52d8a048be944e6d...| null| null| null| null| null| null|536740414| null| refs/heads/gui| null| null| 3|
2489368114| null|277c2dca7505098b4...| null|[[[d3582cd96f7171...| null| 2| null|cf749f4944216ae6a...| null| null| null| null| null| null|536740416| null| refs/heads/master| null| null| 2|
2489368113| null|305bfab9c610dcf74...| null|[[[1e09e78f2250d1...| null| 1| null|f674cbe9d1629d4b4...| null| null| null| null| null| null|536740415| null| refs/heads/master| null| null| 1|
2489368118| null|bda8f8fb4661e972e...| null|[[[67bcad84ab1fac...| null| 0| null|beb1f3edadc32c3d4...| null| null| null| null| null| null|536740420| null| refs/heads/master| null| null| 3|
2489368129| null| null| null| null|Deploy and Manage...| null| null| null| null| master| null| null| null| null| null| user| master2| branch| null|null|
2489368130| null|863240e0b23ad831f...| null|[[[10f94904c14ac5...| null| 1| null|4f24ca5a64048fde0...| null| null| null| null| null| null|536740422| null| refs/heads/master| null| null| 1|
2489368134| null|11f59532b73155573...| null|[[[bfce52bd87de50...| null| 1| null|17ac531e242bf54b1...| null| null| null| null| null| null|536740426| null| refs/heads/master| null| null| 1|
2489368137| null|aedb53b22678c391b...| null|[[[5bbfe2c07a3ef0...| null| 1| null|049abf23362ba353c...| null| null| null| null| null| null|536740428| null| refs/heads/master| null| null| 1|
2489368138| null|38b7187cb1d4f95ee...| null|[[[7fbc091194a948...| null| 1| null|d2b87e31db4c46e27...| null| null| null| null| null| null|536740429| null| refs/heads/master| null| null| 1|
2489368140| null|c77431db7e9a95adf...| null|[[[ae43dfd6142093...| null| 1| null|df2ba5437fc11a326...| null| null| null| null| null| null|536740431| null| refs/heads/master| null| null| 1|
2489368145| null|d2de0eb591002f683...| null|[[[74c17e6b5138e1...| null| 1| null|fd2990e16eb3733ec...| null| null| null| null| null| null|536740433| null| refs/heads/master| null| null| 1|
2489368151|created| null|[, +1 as well. Un...| null| null| null| null| null|[, ,, 5, https://...| null| null| null| null| null| null| null| null| null| null|null|
2489368159| null|42d4e629b4a0c45c3...| null|[[[d462ef6276aa01...| null| 1| null|d6d3e1e7e1866df8d...| null| null| null| null| null| null|

In [38]:
payload_df.describe().show()

+-------+-------------------+-------+--------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+-----------+------------------------------+--------+-----------------+
summary| event_id| action| before| description| distinct_size| head| master_branch| number| push_id|pusher_type| ref|ref_type| size|
+-------+-------------------+-------+--------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+-----------+------------------------------+--------+-----------------+
 count| 82157| 21042| 46919| 8344| 46919| 46919| 8632| 2989| 46919| 9877| 53235| 9877| 46919|
 mean|2.489481050832467E9| null| null| 1616.0|3.6017604808286623| null| 4.04| 839.6731348277016|5.3678933206511223E8| null| 1658233.5480099998| null|4.627379100151325|
 stddev| 59071.733887044255| null| null|4022.7735935067717| 94.33116089738972| null|3.4299854227095485|3175.5900775414816| 25876.688739234687| null| 5545824.27139762| null|47.07197967463356|
 min| 2489368070| added|0000655a12996dd40...| | 0|0000655a12996dd40...| 0.0.8-Z-UIAlpha| 1| 536740396| user| #208| branch| 0|
 max| 2489582585|started|ffffa2087d4f495b7...| 프로 Git, 2판| 10061|ffffa2087d4f495b7...| working| 35415| 536834832| user|自動オープン時のクリア判定修正| tag| 1000|
+-------+-------------------+-------+--------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+-----------+------------------------------+--------+-----------------+

## DataFrame 4: repo_df

In [40]:
repo_df = payload_df.select(F.col("event_id").alias("event_id"),
                            F.col("pull_request.base.repo.name").alias("repo_name"),
                            F.col("pull_request.base.repo.id").alias("repo_id"),
                            F.col("pull_request.base.repo.watchers_count"),
                            F.col("pull_request.base.repo.forks_count"),
                            F.col("pull_request.base.repo.language"),
                            F.col("pull_request.id").alias("pull_req_id")
                            ).dropna()

##### Extract all details related to a repository and the pull requests to the repository from payload_df dataframe

In [42]:
repo_df.show()

+----------+------------------+--------+--------------+-----------+----------+-----------+
 event_id| repo_name| repo_id|watchers_count|forks_count| language|pull_req_id|
+----------+------------------+--------+--------------+-----------+----------+-----------+
2489368161| selfhub|28109544| 0| 5|JavaScript| 26738755|
2489368233| FAKE| 507569| 335| 176| F#| 26738797|
2489368455| selfhub|28109544| 0| 5|JavaScript| 26738755|
2489368612| TextSecure| 2990192| 3242| 963| Java| 26738631|
2489368619|tune-reporting-php|25269682| 0| 1| PHP| 26738805|
2489368625| pdns| 9693534| 293| 101| C++| 26738806|
2489368660| rakudo| 117358| 583| 192| Perl6| 26586595|
2489368673|tune-reporting-php|25269682| 0| 1| PHP| 26738805|
2489368691| tsuru| 3631697| 1294| 136| Go| 26737244|
2489368818| Firmware| 5298790| 226| 719| C++| 26738808|
2489368908| gl3n| 4052471| 59| 28| D| 26738810|
2489368973| flynn|11290232| 3078| 186| Go| 26738812|
2489369017| gabriel|28677119| 0| 2| Java| 26738815|
2489369026| labrador|19842476| 0| 0| PHP| 26738770|
2489369066| labrador|19842476| 0| 0| PHP| 26738770|
2489369117| gabriel|28677119| 0| 2| Java| 26738815|
2489369157| selfhub|28109544| 0| 5|JavaScript| 26738757|
2489369190| Torque3D|28607843| 0| 0| C++| 26738818|
2489369217| candle| 8733622| 1| 1|JavaScript| 26738820|
2489369231| data| 3016562| 2181| 807|JavaScript| 26738712|
+----------+------------------+--------+--------------+-----------+----------+-----------+
only showing top 20 rows

In [43]:
repo_df.printSchema()

root
-- event_id: string (nullable = true)
-- repo_name: string (nullable = true)
-- repo_id: long (nullable = true)
-- watchers_count: long (nullable = true)
-- forks_count: long (nullable = true)
-- language: string (nullable = true)
-- pull_req_id: long (nullable = true)

In [44]:
repo_df.describe().show()

+-------+------------------+-----------------+--------------------+-----------------+-----------------+----------+--------------------+
summary| event_id| repo_name| repo_id| watchers_count| forks_count| language| pull_req_id|
+-------+------------------+-----------------+--------------------+-----------------+-----------------+----------+--------------------+
 count| 3270| 3270| 3270| 3270| 3270| 3270| 3270|
 mean|2.48947800737737E9| 11000.0|1.6165042166666666E7|779.2969418960245|296.1097859327217| null|2.6448366419877674E7|
 stddev| 63212.98541249157| 0.0| 1.00133617450067E7|3812.247302104654|1455.165002442573| null| 1700200.303345197|
 min| 2489368161| -tg-station| 912| 0| 0|AGS Script| 5295589|
 max| 2489582539|zf2-documentation| 28685267| 76097| 29000| XSLT| 26742573|
+-------+------------------+-----------------+--------------------+-----------------+-----------------+----------+--------------------+

###### Cleaning the repo_df by casting the fields to appropriate datatypes

In [46]:
repo_df = repo_df.withColumn("event_id", repo_df["event_id"].cast(T.LongType()))
repo_df.printSchema()

root
-- event_id: long (nullable = true)
-- repo_name: string (nullable = true)
-- repo_id: long (nullable = true)
-- watchers_count: long (nullable = true)
-- forks_count: long (nullable = true)
-- language: string (nullable = true)
-- pull_req_id: long (nullable = true)

##### Aggregating and exploring the repo_df

In [48]:
print("Unique Pull Reqs: ", repo_df.select(F.col("pull_req_id")).distinct().count())
print("Unique Repos: ", repo_df.select(F.col("repo_id")).distinct().count())
print("Unique Events: ", repo_df.select(F.col("event_id")).distinct().count())
print("Unique Repos: ", repo_df.select(F.col("repo_name")).distinct().count())
[ i for i in repo_df.select(F.col("language")).distinct().collect()]

Unique Pull Reqs: 2123
Unique Repos: 1473
Unique Events: 3270
Unique Repos: 1433
Out[75]: [Row(language='C#'),
 Row(language='FORTRAN'),
 Row(language='Makefile'),
 Row(language='Processing'),
 Row(language='Scheme'),
 Row(language='Arc'),
 Row(language='Vala'),
 Row(language='Haxe'),
 Row(language='Perl6'),
 Row(language='Rust'),
 Row(language='JavaScript'),
 Row(language='Emacs Lisp'),
 Row(language='Perl'),
 Row(language='Puppet'),
 Row(language='Game Maker Language'),
 Row(language='Matlab'),
 Row(language='F#'),
 Row(language='C++'),
 Row(language='Groovy'),
 Row(language='TeX'),
 Row(language='Dart'),
 Row(language='OCaml'),
 Row(language='D'),
 Row(language='Julia'),
 Row(language='XSLT'),
 Row(language='C'),
 Row(language='Objective-C'),
 Row(language='Swift'),
 Row(language='TypeScript'),
 Row(language='CSS'),
 Row(language='VimL'),
 Row(language='Elixir'),
 Row(language='Nix'),
 Row(language='AGS Script'),
 Row(language='Haskell'),
 Row(language='AppleScript'),
 Row(language='SQF'),
 Row(language='PowerShell'),
 Row(language='R'),
 Row(language='Scala'),
 Row(language='Clojure'),
 Row(language='Nimrod'),
 Row(language='PHP'),
 Row(language='DM'),
 Row(language='Kotlin'),
 Row(language='Go'),
 Row(language='OpenSCAD'),
 Row(language='Lua'),
 Row(language='Ruby'),
 Row(language='Python'),
 Row(language='PureScript'),
 Row(language='CoffeeScript'),
 Row(language='Gosu'),
 Row(language='Pascal'),
 Row(language='Java'),
 Row(language='Shell'),
 Row(language='Logos'),
 Row(language='Arduino')]

## DataFrame 5: Top500_repos_df

###### Creating a dataframe to contain top500_repos_df by joining repo_df, main_df, org_df and using watchers_count and forks_count fields

In [51]:
print(repo_df.count())
print(main_df.count())

3270
82157

In [52]:
repo_org_df = main_df.select(F.col("repo_id"),
                            F.col("org_id")).dropDuplicates()
repo_org_df.count()

Out[77]: 29109

In [53]:
repo_df.createOrReplaceTempView("repo")
repo_org_df.createOrReplaceTempView("repo_org")
   
temp1 = spark.sql("select o.org_id, r.* from repo r INNER JOIN repo_org o ON r.repo_id == o.repo_id")

In [54]:
temp1.count()

Out[79]: 3270

In [55]:
temp1.createOrReplaceTempView("temp1")
org_df.createOrReplaceTempView("org")

temp2 = spark.sql("select o.login, t.* from temp1 t INNER JOIN org o ON t.org_id == o.id")

In [56]:
temp2.count()

Out[81]: 3270

In [57]:
temp2.show()

+-----------------+--------+----------+------------------+--------+--------------+-----------+----------+-----------+
 login| org_id| event_id| repo_name| repo_id|watchers_count|forks_count| language|pull_req_id|
+-----------------+--------+----------+------------------+--------+--------------+-----------+----------+-----------+
 selfhub|10214048|2489368161| selfhub|28109544| 0| 5|JavaScript| 26738755|
 fsharp| 485415|2489368233| FAKE| 507569| 335| 176| F#| 26738797|
 selfhub|10214048|2489368455| selfhub|28109544| 0| 5|JavaScript| 26738755|
 WhisperSystems| 702459|2489368612| TextSecure| 2990192| 3242| 963| Java| 26738631|
MobileAppTracking| 1920066|2489368619|tune-reporting-php|25269682| 0| 1| PHP| 26738805|
 PowerDNS| 1282630|2489368625| pdns| 9693534| 293| 101| C++| 26738806|
 rakudo| 46671|2489368660| rakudo| 117358| 583| 192| Perl6| 26586595|
MobileAppTracking| 1920066|2489368673|tune-reporting-php|25269682| 0| 1| PHP| 26738805|
 tsuru| 6999787|2489368691| tsuru| 3631697| 1294| 136| Go| 26737244|
 PX4| 2096014|2489368818| Firmware| 5298790| 226| 719| C++| 26738808|
 Missing| -1|2489368908| gl3n| 4052471| 59| 28| D| 26738810|
 flynn| 4973613|2489368973| flynn|11290232| 3078| 186| Go| 26738812|
 TeamGabriel|10360838|2489369017| gabriel|28677119| 0| 2| Java| 26738815|
 Missing| -1|2489369026| labrador|19842476| 0| 0| PHP| 26738770|
 Missing| -1|2489369066| labrador|19842476| 0| 0| PHP| 26738770|
 TeamGabriel|10360838|2489369117| gabriel|28677119| 0| 2| Java| 26738815|
 selfhub|10214048|2489369157| selfhub|28109544| 0| 5|JavaScript| 26738757|
 Missing| -1|2489369190| Torque3D|28607843| 0| 0| C++| 26738818|
 Missing| -1|2489369217| candle| 8733622| 1| 1|JavaScript| 26738820|
 emberjs| 1253363|2489369231| data| 3016562| 2181| 807|JavaScript| 26738712|
+-----------------+--------+----------+------------------+--------+--------------+-----------+----------+-----------+
only showing top 20 rows

#### Aggregating the fields to identify the top500 repositories

In [59]:
top500_repos_df = temp2.where(F.col("login")!= "Missing").groupby("repo_id").agg({'watchers_count':'sum', 'forks_count':'sum'}).withColumnRenamed('sum(watchers_count)','watchers_count').withColumnRenamed('sum(forks_count)', 'forks_count').orderBy('forks_count', ascending = False).orderBy('watchers_count', ascending = False).limit(500)
top500_repos_df.show()

+--------+--------------+-----------+
 repo_id|watchers_count|forks_count|
+--------+--------------+-----------+
 2126244| 380469| 144968|
 206084| 228819| 114143|
 724712| 131473| 25422|
 8514| 122261| 47124|
 486550| 55690| 14536|
 702550| 44748| 27149|
 317757| 42015| 5463|
17165658| 41788| 37055|
 9852918| 40450| 9073|
 5152285| 38783| 8528|
 658518| 38192| 11207|
 4164482| 37245| 14599|
 460078| 33593| 13291|
 167174| 32966| 7753|
 1644196| 29172| 6009|
 458058| 28344| 11067|
 3623050| 28179| 8201|
11225014| 25560| 2760|
 843222| 23410| 13545|
 656494| 21620| 9537|
+--------+--------------+-----------+
only showing top 20 rows

In [60]:
print(type(top500_repos_df))

<class 'pyspark.sql.dataframe.DataFrame'>

###### top10_orgs_df is created by adding organization details to top500_repos_df and aggregating it by organization

In [62]:
top500_repos_df.createOrReplaceTempView("top500_repos_df")
temp2.createOrReplaceTempView("temp2")

temp4 = spark.sql("select t2.login, t1.* from top_repo_df t1 INNER JOIN temp2 t2 ON t1.repo_id == t2.repo_id")

In [63]:
top10_orgs_df = temp4.groupby("login").agg({'repo_id':'count'}).withColumnRenamed('count(repo_id)','popular_repo_count').orderBy('popular_repo_count', ascending = False).limit(10)
top10_orgs_df.show()

+--------------------+------------------+
 login|popular_repo_count|
+--------------------+------------------+
 Coursemology| 74|
 cachethq| 43|
 tgstation| 34|
 blueboxgroup| 23|
 CleverRaven| 22|
 rspec| 20|
 PCMDI| 19|
D-Programming-Lan...| 19|
 apache| 17|
 rust-lang| 16|
+--------------------+------------------+

###### Dumping the DataFrames as csv files to create visualizations in Tableau.

In [65]:
main_df.write.csv('/dbfs/mnt/main.csv')
repo_df.write.csv('/dbfs/mnt/repo.csv')
org_df.write.csv('/dbfs/mnt/org.csv')
top500_repos_df.write.csv('/dbfs/mnt/top500_repos.csv')
top10_orgs_df.write.csv('/dbfs/mnt/top10_orgs.csv')

In [66]:
main_df.createOrReplaceTempView("main_table")
repo_df.createOrReplaceTempView("repo_table")
org_df.createOrReplaceTempView("org_table")
top500_repos_df.createOrReplaceTempView("top500_repos_table")
top10_orgs_df.createOrReplaceTempView("top10_orgs_table")

#### Question 1: 

###### There are close to 20+ types of GitHub events possible (as per the GitHub archive data). In the data we analyzed, there are 14 distinct events that happened over the span of 12 hours. But what are the top 5 events by volume? Does the top 5 events change across org_users and individual_users?

#### Question 2: 

###### There are two types of user segments in GitHub-Archive data (The archive doesn't contain any details about GitHub Enterprise). Org_users who are a part of an organization pay for collaborating with their team through GitHub. Individual users are other type of users who do not pay for the GitHub service. 

###### How different is the event activity volume amongst org_users and individual_users? Do org_users trigger more events or individual users?

#### Question 3: 

###### What are the most popular repositories in GitHub? How do we decide the popular repositories? Although not complete, we use "watchers_count" and "forks_count" as a measure of popular repositories. And, which organizations maintain those repositories?